# 第四步：处理结果

以下介绍如何对优化结果进行处理以进行评估和可视化。

1. 准备以下参数：

|参数 |说明 |数值 |
|--- |--- |--- |
|method | QUBO问题的退火方法|'dwave-sa'：使用Ocean工具包中的模拟退火器<br>'dwave-qa'：使用量子退火器|
|raw_path|原始分子文件的路径| 本示例使用的是'./molecule-data/117_ideal.mol2'|
|data_path|处理后的分子文件的路径| 本示例使用的是'./qmu_117_ideal_data_latest.mol2'|
|bucket |存储结果的 S3 存储桶 | - |
|prefix | S3 存储桶中的文件夹名称 | - |
|task_id |量子退火任务id| 本示例使用的是'2b5a3b05-1a0e-443a-852c-4ec422a10e59' |

2. 使用SA的 **ResultParser**对象进行处理。

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [27]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 129.5549328327179


In [28]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-sa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-sa_20220323-14.json


In [29]:
sa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 11}}}

在第一个代码块中，可以看到SA的**local time**
大约为 174 秒。
使用 **generate_optimize_pts()** 方法，最终的 3D
展开后的点会生成并保存为json文件和mol2文件。
最后一个代码块显示优化结果，也存储在 json 文件中。从结果看出体积增加
1.0212倍。 

3. 使用 QA 进行处理：

In [30]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/results.json


time for dwave-qa: 
     local time is 11.894113779067993,
     task time is 3.327, 
     qpu total time is 0.144422, 
     qpu access time is 0.129189



运行 QA 的时间指标有很多种。
该任务的**local time**为 7.7 秒，即调用 API 和
得到退火结果之间的时间。 **task time**  是来自存放在桶里面的json 文件的指标。**qpu total time**和**qpu access time**表示
在 QPU 中运行的实际时间。详情可参考[操作与时序](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)。

In [31]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-qa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-qa_20220323-14.json


In [32]:
qa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 1}}}


同理，优化后的结果被保存为本地json和mol2文件。结果表明QA获得了
1.021倍体积增加。

4. 打开文件夹以获取优化的结果。

如果执行更多实验，更多带有时间戳的结果会存储在这里。

![optimize-results](../../../images/optimize-results.png)

 图13：优化结果

5. 使用以下方法查看原始的和展开的分子形态。

In [37]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal.mol2', description='mol'), IntSlider(value=800, de…

In [38]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal_dwave-qa_20220323-14.mol2', description='mol'), In…

6. （可选）将mol2文件上传到
[Protein Data Bank online viewer tool](https://www.rcsb.org/3d-view) 
查看结果:

![visual](../../../images/visualization.png)

图14：可视化